In [ ]:
#!pip install ultralytics opencv-python pillow # OBS: Coloquei Early Stopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from PIL import Image
import yaml
import numpy as np
import cv2
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import timm
from sklearn.metrics import accuracy_score, f1_score
from itertools import product
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
path = os.getcwd()
print(path)

#os.chdir(path)
#file_log = open(path + "/mensagem_final_classificar_V2.txt", "a")

/content


In [ ]:
# === Configurações Gerais ===
NUM_CLASSES = 15
INPUT_SIZE = 224  # conforme modelo Small treinado em 224×224
BATCH_SIZE = 32 # Padrão: 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DATASET_ROOT = "/content/drive/MyDrive/TCC/Datasets/Imagens Folhas/Especies" # Dataset de folhas usado
DATASET_ROOT = "/content/drive/MyDrive/TCC/Datasets/Imagens tronco/EspeciesCascas" # Dataset de cascas usado

In [ ]:
#  ===Dataset customizado ===
class ImageFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Assumindo estrutura: root_dir/class_x/imagename.jpg
        self.samples = []
        classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name:i for i,cls_name in enumerate(classes)}
        for cls_name in classes:
            cls_path = os.path.join(root_dir, cls_name)
            if not os.path.isdir(cls_path):
                continue
            for fname in os.listdir(cls_path):
                if fname.lower().endswith(('.png','.jpg','.jpeg','.bmp')):
                    self.samples.append((os.path.join(cls_path, fname), self.class_to_idx[cls_name]))
        # embaralhar?
        np.random.seed(42)
        np.random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        # leitura via OpenCV
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # redimensionar
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
        # converter para float32 e normalizar [0,1]
        img = img.astype(np.float32) / 255.0
        # talvez normalização adicional conforme modelo (media/std)
        # usando valores padrão ImageNet
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        img = (img - mean) / std
        # mudar de H×W×C para C×H×W
        img = np.transpose(img, (2,0,1))
        img_tensor = torch.from_numpy(img)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor, label_tensor

In [ ]:
def stratified_split(dataset, test_split=0.1, valid_split=0.2, seed=42):
    """
    Divide o dataset em treino, validação e teste mantendo a proporção das classes.

    Args:
        dataset: Dataset com atributo 'samples' [(path, label), ...]
        test_split: Proporção do teste em relação ao total (padrão: 0.1 = 10%)
        valid_split: Proporção da validação em relação ao (treino+validação) (padrão: 0.2 = 20%)
        seed: Seed para reprodutibilidade

    Returns:
        train_ds, valid_ds, test_ds: Subsets estratificados

    Nota: Com os valores padrão, a divisão real é:
        - Treino: 72% do total
        - Validação: 18% do total
        - Teste: 10% do total
    """
    # Extrair os rótulos
    labels = [label for _, label in dataset.samples]

    # Primeiro, separar Teste
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_split, random_state=seed)
    train_valid_idx, test_idx = next(sss1.split(np.zeros(len(labels)), labels))

    # Agora, separar Validação dentro do conjunto de treino+validação
    labels_train_valid = np.array(labels)[train_valid_idx]
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=valid_split, random_state=seed)
    train_idx, valid_idx = next(sss2.split(np.zeros(len(labels_train_valid)), labels_train_valid))

    # Reindexar para o dataset original
    train_idx = np.array(train_valid_idx)[train_idx]
    valid_idx = np.array(train_valid_idx)[valid_idx]

    # Criar Subsets
    train_ds = torch.utils.data.Subset(dataset, train_idx)
    valid_ds = torch.utils.data.Subset(dataset, valid_idx)
    test_ds  = torch.utils.data.Subset(dataset, test_idx)

    return train_ds, valid_ds, test_ds

In [ ]:
# === Função para treinar por uma época ===
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(loader, desc="Train"):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

# === Função para avaliar no conjunto de validação/teste ===
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Eval"):
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(imgs)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average='weighted')
    return acc, f1

# === Função para montar o modelo adaptado para 15 classes ===
def create_model(num_classes=NUM_CLASSES, pretrained=True):
    # Carregar modelo base
    model = timm.create_model('vit_tiny_patch16_224', pretrained=pretrained)

    # Primeiro, descongela tudo
    for param in model.parameters():
        param.requires_grad = True

    # Agora CONGELA apenas os blocos iniciais (features básicas)
    for name, param in model.named_parameters():
        # Congela APENAS 2 blocos iniciais (17%), NÃO congela patch_embed
        if any(x in name for x in ["blocks.0", "blocks.1"]):
            param.requires_grad = False

    model.reset_classifier(num_classes=num_classes)
    return model.to(DEVICE)

In [ ]:
# === Função principal para rodar Grid Search sobre epochs e learning rate ===
def run_grid_search(dataset_root, lr_list, epoch_list, valid_split=0.2, test_split=0.1, patience=2):
    # carregar dataset completo
    #global full_ds, train_ds, valid_ds, test_ds
    full_ds = ImageFolderDataset(dataset_root)
    total_len = len(full_ds)
    test_len  = int(total_len * test_split)
    valid_len = int((total_len - test_len) * valid_split)
    train_len = total_len - test_len - valid_len
    #train_ds, valid_ds, test_ds = random_split(full_ds, [train_len, valid_len, test_len], generator=torch.Generator().manual_seed(42))
    train_ds, valid_ds, test_ds = stratified_split(full_ds, test_split=test_split, valid_split=valid_split, seed=42)

    print(f"\nDataset total: {total_len} imagens")
    print(f"Classes detectadas ({len(full_ds.class_to_idx.keys())}): {full_ds.class_to_idx.keys()}")
    print(f"Treino: {train_len} | Validação: {valid_len} | Teste: {test_len}")

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    best_config = None
    best_f1 = 0.0 # Variável usada para o teste final para saber qual é a melhor configuração
    best_val_f1 = 0.0 # Variável usada para o Early Stopping
    early_stop = False
    patience_counter = 0

    for lr, epochs in product(lr_list, epoch_list):
        print(f"\n==== Training with lr={lr}, epochs={epochs} ====")
        best_val_f1 = 0.0 # Reseta sempre que começa uma nova configuração de hiperparâmetros
        model = create_model(num_classes=NUM_CLASSES, pretrained=True)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # === Novo: tentar carregar último checkpoint existente ===
        #ckpt_dir = "checkpointsViT" # Para salvar localmente, não no Drive
        ckpt_dir = "/content/drive/MyDrive/TCC/Datasets/checkpointsViT"
        os.makedirs(ckpt_dir, exist_ok=True)

        # Procura checkpoints com o padrão "ViT_Tiny_epochX_eY_lrZ.pt" (X = época atual, Y = número de épocas máximas, Z = Learning Rate)
        ckpt_files = [f for f in os.listdir(ckpt_dir) if f"e{epochs:.0e}_lr{lr:.0e}" in f]
        last_epoch = 0
        if ckpt_files:
            # Ordena checkpoints por número da época
            ckpt_files.sort(key=lambda x: int(x.split("_epoch")[1].split("_")[0]))
            last_ckpt = os.path.join(ckpt_dir, ckpt_files[-1])
            print(f"Checkpoint detectado: {last_ckpt} — retomando treinamento...")
            checkpoint = torch.load(last_ckpt, map_location=DEVICE)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            last_epoch = checkpoint['epoch']
            # Testa se já chegou no final do treinamento
            if last_epoch == epochs:
                print("Treinamento já foi finalizado!")
                loss, acc_val, f1_val = checkpoint['loss'], checkpoint['val_acc'], checkpoint['val_f1']
                print(f"Epoch {last_epoch}/{epochs} - loss: {loss:.6f}, val_acc: {acc_val:.4f}, val_f1: {f1_val:.4f}")
            else:
                print(f"Retomando a partir da época {last_epoch+1}/{epochs} (lr={lr})")
        else:
            print("Nenhum checkpoint anterior encontrado, começando do zero.")

        for epoch in range(last_epoch, epochs): # OBS: Se já tiver terminado, nem entra nesse loop
            loss = train_one_epoch(model, train_loader, criterion, optimizer)
            acc_val, f1_val = evaluate(model, valid_loader)
            print(f"Epoch {epoch+1}/{epochs} - loss: {loss:.6f}, val_acc: {acc_val:.4f}, val_f1: {f1_val:.4f}")

            # === Early Stopping por val_f1 ===
            if f1_val > best_val_f1:
                best_val_f1 = f1_val
                patience_counter = 0
                best_model_state = model.state_dict()
                best_epoch = epoch + 1
            else:
                patience_counter += 1
                print(f"Contador do Early stopping: {patience_counter}/{patience} - Melhor F1-Score anterior: {best_val_f1}")
                if patience_counter >= patience:
                    print(f"Parando antecipadamente na época {epoch+1}. Melhor val_f1: {best_val_f1:.4f}")
                    early_stop = True

        # === salvar checkpoint por época ===
            #ckpt_dir = "checkpointsViT" # Para salvar localmente, não no Drive
            ckpt_dir = "/content/drive/MyDrive/TCC/Datasets/checkpointsViT"
            os.makedirs(ckpt_dir, exist_ok=True)
            if early_stop: # Salva a época atual como se fosse a última caso
              ckpt_path = os.path.join(ckpt_dir, f"ViT_Tiny_epoch{epochs}_e{epochs:.0e}_lr{lr:.0e}.pt")
              current_epoch = epochs
              #saved_model_state = best_model_state # Se for parar, salva usando o melhor modelo que tinha achado
            else:
              current_epoch = epoch + 1
              ckpt_path = os.path.join(ckpt_dir, f"ViT_Tiny_epoch{current_epoch}_e{epochs:.0e}_lr{lr:.0e}.pt")
              #saved_model_state = model.state_dict()
            torch.save({
                'epoch': current_epoch,
                'model_state_dict': model.state_dict(), #saved_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': acc_val,
                'val_f1': f1_val,
                'loss': loss,
                'lr': lr
            }, ckpt_path)
            print(f"Checkpoint salvo: {ckpt_path}")
            ckpt_path = os.path.join(ckpt_dir, f"ViT_Tiny_epoch{epoch}_e{epochs:.0e}_lr{lr:.0e}.pt")
            if os.path.exists(ckpt_path):
              os.remove(ckpt_path) # Apaga o checkpoint anterior
              print(f"Checkpoint deletado: {ckpt_path}")
            if early_stop:
              break # Early Stopping se o F1 Score não melhorar em 5 iterações

        # avaliar no conjunto de validação final
        early_stop = False
        patience_counter = 0 # Resetando a parte do Early Stopping
        if f1_val > best_f1:
            best_f1 = f1_val
            best_config = {'lr': lr, 'epochs': epochs, 'model_state': model.state_dict()}
            print(f"*** New best config: lr={lr}, epochs={epochs}, val_f1={f1_val:.4f} ***")

    # Depois de escolher melhor configuração, reavaliar no test set
    print(f"\nBest config found: {best_config['lr']=}, {best_config['epochs']=}, best_val_f1={best_f1:.4f}")
    # carregar o melhor modelo
    best_model = create_model(num_classes=NUM_CLASSES, pretrained=False)
    best_model.load_state_dict(best_config['model_state'])
    best_model.to(DEVICE)

    acc_test, f1_test = evaluate(best_model, test_loader)
    print(f"Test set - acc: {acc_test:.6f}, f1: {f1_test:.6f}")

    # === Novo: salvar pesos finais do melhor modelo ===
    #final_dir = "main_weights" # Para salvar localmente, não no Drive
    final_dir = "/content/drive/MyDrive/TCC/Datasets/main_weights"
    os.makedirs(final_dir, exist_ok=True)
    final_path = os.path.join(final_dir, "ViT_Tiny_best.pt")
    torch.save(best_model.state_dict(), final_path)
    print(f"Pesos finais salvos em: {final_path}")

    return best_config, (acc_test, f1_test)

In [ ]:
# === Função Main ===
if __name__ == "__main__":
    try:
        print("\n--------------- Treinamento do modelo ViT-Tiny ---------------\nInício...")
        # Defina aqui o caminho para o diretório de imagens
        dataset_root = DATASET_ROOT
        # Defina lista de valores para hyperparâmetros
        lr_list    = [5e-4, 1e-4, 5e-5, 1e-5]
        epoch_list = [10, 20, 30]
        # Rodar grid
        best_config, test_metrics = run_grid_search(dataset_root, lr_list, epoch_list, valid_split=0.2, test_split=0.1)
        print("Finished. Test metrics:", test_metrics)
        print("...Fim\n")
    except KeyboardInterrupt:
        print("Programa encerrado via terminal...")


--------------- Treinamento do modelo ViT-Tiny ---------------
Início...

Dataset total: 467 imagens
Classes detectadas (15): dict_keys(['Abacateiro', 'Araca', 'Brinco de Indio', 'Cajueiro', 'Carvalho', 'Caterete', 'Cerejeira', 'Coite', 'Fruta do conde', 'Grevilha', 'Jambolao', 'Laranja Champanhe', 'Louro Pardo', 'Pau Brasil', 'Peroba Rosa'])
Treino: 337 | Validação: 84 | Teste: 46

==== Training with lr=0.0005, epochs=10 ====


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Checkpoint detectado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch10_e1e+01_lr5e-04.pt — retomando treinamento...
Treinamento já foi finalizado!
Epoch 10/10 - loss: 0.033917, val_acc: 0.9405, val_f1: 0.9382
*** New best config: lr=0.0005, epochs=10, val_f1=0.9382 ***

==== Training with lr=0.0005, epochs=20 ====
Checkpoint detectado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch20_e2e+01_lr5e-04.pt — retomando treinamento...
Treinamento já foi finalizado!
Epoch 20/20 - loss: 0.055910, val_acc: 0.8571, val_f1: 0.8536

==== Training with lr=0.0005, epochs=30 ====
Checkpoint detectado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch30_e3e+01_lr5e-04.pt — retomando treinamento...
Treinamento já foi finalizado!
Epoch 30/30 - loss: 0.008250, val_acc: 0.8810, val_f1: 0.8794

==== Training with lr=0.0001, epochs=10 ====
Checkpoint detectado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch10_e1e+01_lr1e-04.pt — retoman

Eval: 100%|██████████| 3/3 [00:26<00:00,  8.88s/it]


...

Epoch 13/30 - loss: 0.316554, val_acc: 0.7738, val_f1: 0.7749
Contador do Early stopping: 1/2 - Melhor F1-Score anterior: 0.7877151287865574
Checkpoint salvo: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch13_e3e+01_lr1e-05.pt
Checkpoint deletado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch12_e3e+01_lr1e-05.pt


Train:   0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Eval: 100%|██████████| 3/3 [00:13<00:00,  4.52s/it]


Epoch 14/30 - loss: 0.258852, val_acc: 0.7857, val_f1: 0.7871
Contador do Early stopping: 2/2 - Melhor F1-Score anterior: 0.7877151287865574
Parando antecipadamente na época 14. Melhor val_f1: 0.7877
Checkpoint salvo: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch30_e3e+01_lr1e-05.pt
Checkpoint deletado: /content/drive/MyDrive/TCC/Datasets/checkpointsViT/ViT_Tiny_epoch13_e3e+01_lr1e-05.pt

Best config found: best_config['lr']=5e-05, best_config['epochs']=20, best_val_f1=0.9643


Eval:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Eval: 100%|██████████| 2/2 [00:26<00:00, 13.29s/it]


Test set - acc: 0.978723, f1: 0.978723
Pesos finais salvos em: /content/drive/MyDrive/TCC/Datasets/weights/ViT_Tiny_best.pt
Finished. Test metrics: (0.9787234042553191, 0.9787234042553191)
...Fim

